In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.preprocessing import StandardScaler
from tqdm.keras import TqdmCallback
import os

drive.mount('/content/drive')

df = pd.read_csv('/content/dataSet.csv')

Mounted at /content/drive


In [ ]:
# 범주형 데이터 인코딩
le_gender = LabelEncoder()
df['gender'] = le_gender.fit_transform(df['gender'])

le_experience = LabelEncoder()
df['experience'] = le_experience.fit_transform(df['experience'])

# 피처와 타겟 분리
X = df[['altitude', 'gender', 'age', 'distance', 'experience', 'average_time']]
y = df['time']

# 데이터 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 정규화 파라미터 저장
np.savetxt('/content/scaler_mean.txt', scaler.mean_)
np.savetxt('/content/scaler_scale.txt', scaler.scale_)

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 학습
model.fit(X_train, y_train, epochs=50, verbose=0, callbacks=[TqdmCallback(verbose=1)])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

In [ ]:
# 모델 평가
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# 예측
predictions = model.predict(X_test)
print(f'Predictions: {predictions.flatten()}')
print(f'Actual Values: {y_test.values}')

# 모델 저장
model.save('/content/hiking_time_prediction_model.keras')

# TensorFlow Lite 모델로 변환
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# .tflite 파일로 저장
with open('/content/hiking_time_prediction_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("모델과 정규화 파라미터가 Colab 로컬 경로에 저장되었습니다.")
!cp /content/hiking_time_prediction_model.tflite '{drive_path}/hiking_time_prediction_model.tflite'

6250/6250 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - loss: 357.3108
Test Loss: 357.112548828125
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step
Predictions: [ 84.17891   41.29824   18.354225 ... 201.88824   48.40655   72.63735 ]
Actual Values: [ 99.05597116  35.07343891  20.4701569  ... 237.11743339  29.13246385
  65.31167094]
Saved artifact at '/tmp/tmp79vbp3yx'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 6), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  136719041608016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136719041607840: TensorSpec(shape=(), dtype=tf.resource, name=None)
모델과 정규화 파라미터가 Colab 로컬 경로에 저장되었습니다.
cp: cannot create regular file '{drive_path}/hiking_time_prediction_model.tflite': No such file or directory


In [ ]:
# 예시 인자 테스트
def predict_hiking_time(input_features):
    # 정규화 파라미터 로드
    scaler_mean = np.loadtxt('/content/scaler_mean.txt')
    scaler_scale = np.loadtxt('/content/scaler_scale.txt')

    # 입력 데이터 정규화
    input_features = np.array(input_features).reshape(1, -1)
    input_features_scaled = (input_features - scaler_mean) / scaler_scale

    # 모델 로드
    loaded_model = tf.keras.models.load_model('/content/hiking_time_prediction_model.keras')

    # 예측 수행
    prediction = loaded_model.predict(input_features_scaled)
    return prediction[0, 0]

# 예시 인자 (altitude, gender, age, distance, experience, average_time)
example_input = [300, 1, 28, 2000, 1, 300]

# 예측
predicted_time = predict_hiking_time(example_input)
print(f'Predicted Hiking Time: {predicted_time:.2f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
Predicted Hiking Time: 540.93


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 예측
predictions = model.predict(X_test)

# Mean Absolute Error
mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error (MAE): {mae:.2f}')

# Mean Squared Error
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error (MSE): {mse:.2f}')

# Root Mean Squared Error
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')

# R-squared Score
r2 = r2_score(y_test, predictions)
print(f'R-squared Score (R²): {r2:.2f}')

6250/6250 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step
Mean Absolute Error (MAE): 12.20
Mean Squared Error (MSE): 357.11
Root Mean Squared Error (RMSE): 18.90
R-squared Score (R²): 0.90


In [ ]:
print(le_gender.classes_)
print(le_experience.classes_)

['female' 'male']
['high' 'low' 'medium']


In [ ]:
print("Gender Mapping:")
print(dict(enumerate(le_gender.classes_)))  # {0: 'female', 1: 'male'}

print("Experience Mapping:")
print(dict(enumerate(le_experience.classes_)))  # {0: 'high', 1: 'low', 2: 'medium'}

Gender Mapping:
{0: 'female', 1: 'male'}
Experience Mapping:
{0: 'high', 1: 'low', 2: 'medium'}
